В качестве домашки
1. Попробуйте обучить три сети представленные в тетради.
(Разморозкой слоев заниматься не надо, разберем разморозку на уроке.)
Сохраните модели.

2. Напишите функцию принимающую изображение и дающую прогноз (кот /собака) с использованием всех трех моделей.

In [ ]:
from google.colab import drive
from google.colab import files

In [ ]:
!mkdir GD

In [ ]:
drive.mount('/GD',force_remount=True)

Mounted at /GD


In [ ]:
!cp -r /GD/'My Drive'/"data"/PetImages.zip .

In [ ]:
!ls /GD/'My Drive'/"data"/

PetImages.zip  test_labels.csv	train_labels.csv


In [ ]:
!cd  /GD/'My Drive'/"data"/

In [ ]:
!unzip PetImages.zip


In [ ]:
!cd

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers

In [ ]:
!ls 

GD  PetImages  PetImages.zip  sample_data


In [ ]:
!mkdir PetImages/validation_data

In [ ]:
!mkdir PetImages/validation_data/Dog

In [ ]:
!mkdir PetImages/validation_data/Cat

In [ ]:
!ls PetImages/validation_data

Cat  Dog


## разделение данных на тестовое и обучающее множество

In [ ]:
import os, shutil
import random as rn

In [ ]:
train_dirs = ['PetImages/train/Dog', 
              'PetImages/train/Cat']

test_dirs =  ['PetImages/validation_data/Dog', 
              'PetImages/validation_data/Cat']

In [ ]:
# Страшный код

for i in range(len(train_dirs)):
    fnames = os.listdir(train_dirs[i])
    toCopy = len(fnames)//10

    filesToCopy = []
    for j in range(toCopy):
        filesToCopy.append(rn.choice(fnames))
    
    # cOPY
         
    for fi in filesToCopy:
        src = os.path.join(train_dirs[i],fi)
        dst = os.path.join(test_dirs[i], fi)
        shutil.copyfile(src,dst)

    
    for fi in filesToCopy:
        try:
            src = os.path.join(train_dirs[i],fi)
            os.remove(src)
            print(src+' deleted')
            #time.sleep(1)
        except:
            pass


In [ ]:
# осмотр получившихся директорий

In [ ]:
!ls PetImages/validation_data/Dog

## подготовка генераторов

In [ ]:
# Воспоминание о генераторах

In [ ]:
def int_generator ():
    i = 0
    while True:
        yield i
        i += 1
        

In [ ]:
j = 0
for item in int_generator():
    print (item)
    j += 1
    if j > 10:
        break

In [ ]:
# взрослый генератор

In [ ]:
Pic_Y, Pic_X = 180, 180
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18517 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
# Обучение модели с 0

model = models.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))

model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 41, 41, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 20, 20, 128)      0

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.h5",
        save_best_only=True,
        monitor="val_accuracy")]

In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/10
290/290 [==============================] - ETA: 0s - loss: 0.6609 - accuracy: 0.5962

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


290/290 [==============================] - 97s 291ms/step - loss: 0.6609 - accuracy: 0.5962 - val_loss: 0.5556 - val_accuracy: 0.7118
Epoch 2/10
290/290 [==============================] - 82s 284ms/step - loss: 0.5426 - accuracy: 0.7279 - val_loss: 0.4672 - val_accuracy: 0.7823
Epoch 3/10
290/290 [==============================] - 83s 285ms/step - loss: 0.4542 - accuracy: 0.7892 - val_loss: 0.3917 - val_accuracy: 0.8173
Epoch 4/10
290/290 [==============================] - 83s 286ms/step - loss: 0.3774 - accuracy: 0.8310 - val_loss: 0.3630 - val_accuracy: 0.8322
Epoch 5/10
290/290 [==============================] - 88s 302ms/step - loss: 0.3184 - accuracy: 0.8639 - val_loss: 0.3191 - val_accuracy: 0.8589
Epoch 6/10
290/290 [==============================] - 82s 284ms/step - loss: 0.2657 - accuracy: 0.8868 - val_loss: 0.4309 - val_accuracy: 0.8380
Epoch 7/10
290/290 [==============================] - 83s 286ms/step - loss: 0.2160 - accuracy: 0.9095 - val_loss: 0.2482 - val_accuracy: 0.8

In [ ]:
!ls

In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp convnet_from_scratch.h5 /GD/'My Drive'/convnet_from_scratch.h5

In [ ]:
!ls /GD/'My Drive'/

 20211031_164435.mp4	   V5-29-8-2019.skp
 20211110_161321.mp4	   VGG_FIX_model.h5
 augmentation_model.h5	  'מסגריה בת ים לוח חשמל001.pdf'
'Colab Notebooks'	  'מסגריה בת ים לוח חשמל002.pdf'
 convnet_from_scratch.h5  'מסגריה בת ים לוח חשמל003.pdf'
 data			  'מסגריה בת ים לוח חשמל004.pdf'
 images			  'מסגריה בת ים לוח חשמל005.pdf'
 REKA-light-Layout1.pdf   'מסגריה בת ים לוח חשמל006.pdf'
 REKA-power-Model.pdf	  'מסגריה בת ים לוח חשמל007.pdf'


## Добавляем расширение данных 

In [ ]:
model_2 = models.Sequential()

model_2.add(layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape = (Pic_Y, Pic_X, 3)))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=64, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=128, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))

model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))
model_2.add(layers.MaxPooling2D(pool_size=2))
model_2.add(layers.Conv2D(filters=256, kernel_size=3, activation="relu"))

model_2.add(layers.Flatten())
model_2.add(layers.Dense(1, activation="sigmoid"))


In [ ]:
model_2.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
# Новый интересный объект
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="augmentation_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 25,
                                   shear_range = 0.15,
                                   zoom_range = 0.3,
                                   fill_mode = 'nearest')
                            
train_generator = train_datagen.flow_from_directory('PetImages/train',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 18512 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
                            
test_generator = test_datagen.flow_from_directory('PetImages/test',
                                                   target_size = (Pic_Y, Pic_X),
                                                   batch_size = 64,
                                                   class_mode = 'binary')

Found 4542 images belonging to 2 classes.


In [ ]:
history = model_2.fit(train_generator, epochs=10, validation_data=test_generator,
    callbacks=callbacks)

Epoch 1/10
290/290 [==============================] - 199s 680ms/step - loss: 0.6623 - accuracy: 0.5996 - val_loss: 0.6871 - val_accuracy: 0.6513
Epoch 2/10
290/290 [==============================] - 199s 686ms/step - loss: 0.5773 - accuracy: 0.6973 - val_loss: 0.5054 - val_accuracy: 0.7512
Epoch 3/10
290/290 [==============================] - 199s 685ms/step - loss: 0.5179 - accuracy: 0.7458 - val_loss: 0.4657 - val_accuracy: 0.7796
Epoch 4/10
290/290 [==============================] - 202s 697ms/step - loss: 0.4630 - accuracy: 0.7831 - val_loss: 0.4450 - val_accuracy: 0.7922
Epoch 5/10
290/290 [==============================] - 196s 676ms/step - loss: 0.4201 - accuracy: 0.8051 - val_loss: 0.3782 - val_accuracy: 0.8285
Epoch 6/10
290/290 [==============================] - 198s 682ms/step - loss: 0.3788 - accuracy: 0.8300 - val_loss: 0.3088 - val_accuracy: 0.8655
Epoch 7/10
290/290 [==============================] - 196s 677ms/step - loss: 0.3487 - accuracy: 0.8459 - val_loss: 0.3031 -

In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp augmentation_model.h5 /GD/'My Drive'/augmentation_model.h5

In [ ]:
!ls

augmentation_model.h5	 GD	    PetImages.zip
convnet_from_scratch.h5  PetImages  sample_data


## Перенос знаний!

In [ ]:
# закгрузка сверточных слоев VGG16

conv_base = tf.keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Заморозка сверточных слоев
conv_base.trainable = False

In [ ]:
# Конструируем модель
model_3 = models.Sequential()
model_3.add(conv_base)
model_3.add(layers.Flatten())
model_3.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_3.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(
        filepath="VGG_FIX_model.h5",
        save_best_only=True,
        monitor="val_loss")]

In [ ]:
# фитим на тех же генераторах то есть с расширением данных
history = model_3.fit(train_generator, epochs=5, validation_data=test_generator, 
    callbacks=callbacks)

Epoch 1/5
290/290 [==============================] - 218s 730ms/step - loss: 0.3946 - accuracy: 0.8177 - val_loss: 0.2023 - val_accuracy: 0.9139
Epoch 2/5
290/290 [==============================] - 207s 712ms/step - loss: 0.2883 - accuracy: 0.8765 - val_loss: 0.2042 - val_accuracy: 0.9148
Epoch 3/5
290/290 [==============================] - 207s 713ms/step - loss: 0.2659 - accuracy: 0.8853 - val_loss: 0.1835 - val_accuracy: 0.9234
Epoch 4/5
290/290 [==============================] - 206s 712ms/step - loss: 0.2553 - accuracy: 0.8922 - val_loss: 0.3271 - val_accuracy: 0.8672
Epoch 5/5
290/290 [==============================] - 206s 712ms/step - loss: 0.2419 - accuracy: 0.8975 - val_loss: 0.2568 - val_accuracy: 0.8954


In [ ]:
# Копирование сохраненой модели на гугл диск.
!cp VGG_FIX_model.h5 /GD/'My Drive'/VGG_FIX_model.h5

In [ ]:
!ls /content/PetImages/validation_data/Dog

In [ ]:
!ls

GD  PetImages  PetImages.zip  sample_data


In [ ]:
from PIL import Image
import numpy as np


def predict_image(image_path):
    # Загрузка изображения и подготовка его для подачи в модели
    img = Image.open(image_path)
    img = img.resize((Pic_Y, Pic_X)) # изменение размера изображения
    img = np.array(img) / 255.0 # нормализация значений пикселей
    img = np.expand_dims(img, axis=0) # добавление измерения пакета
    
    # Загрузка каждой из трех моделей
    model1 = tf.keras.models.load_model('/GD/MyDrive/augmentation_model.h5')
    model2 = tf.keras.models.load_model('/GD/My Drive/augmentation_model.h5')
    model3 = tf.keras.models.load_model('/GD/My Drive/VGG_FIX_model.h5')
    
    # Предсказание на входном изображении для каждой модели
    pred1 = model1.predict(img)
    pred2 = model2.predict(img)
    pred3 = model3.predict(img)
    
    # Принятие окончательного решения на основе предсказаний всех трех моделей
    final_pred = np.mean([pred1, pred2, pred3], axis=0)
    if final_pred > 0.5:
        return 'Dog'
    else:
        return 'Cat'


In [ ]:
predict_image('/content/PetImages/validation_data/Dog/3120.jpg')

1/1 [==============================] - 1s 906ms/step


'Dog'